In [1]:
import sys
import os
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import random
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
dataDir = thisDir + 'data/mvpa_data/'
outDir = thisDir + 'output/mL/results/ridge/acc/'
import classification
subDir=thisDir+'output/subNetwork/'
randDir=thisDir+'output/randomIdx/results/ridge/acc/'
netList=['unassign','default', 'visual','fp','dan','van','salience','co','sm','sm-lat', 'auditory','pmn','pon']
feature=[14808,10824,8736, 4620,5264,3151, 484, 4060, 2375,316, 564,45,21]
netRoi=dict([('unassign',14808),('default', 10824),('visual',8736),('fp', 4620),('dan',5264),('van',3151),('salience', 484),('co', 4060),('sm', 2375),('sm-lat', 316),('auditory', 564),('pmn',45),('pon',21)])
taskList=['mixed','mem','motor']
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']


permSS=pd.read_csv(thisDir+'output/mL/results/ridge/permutation/SS/acc.csv')
pairCV=pd.read_csv(thisDir+'output/mL/results/ridge/permutation/CV/acc.csv')
permDS=pd.read_csv(thisDir+'output/mL/results/ridge/permutation/DS/acc.csv')
permBS=pd.read_csv(thisDir+'output/mL/results/ridge/permutation/BS/acc.csv')



#adding thinner lines to different task analysis figure
BS_startIndex=0
BS_endIndex=336
iterations=0
pairBS=pd.DataFrame()
while BS_startIndex<336001:
    BS_iterat=permBS[BS_startIndex:BS_endIndex]
    for task in taskList:
        for test_task in taskList:
            if task==test_task:
                continue
            else:
                tmp=BS_iterat[(BS_iterat.train_task==task)&(BS_iterat.test_task==test_task)]
                #iterating through each permutationt to recreate the previous plots
                tmp_mu=tmp.groupby(['train_sub']).mean()
                tmp_mu['train_task']=task
                tmp_mu['test_task']=test_task
                tmp_mu['iteration']=iterations
                pairBS=pd.concat([pairBS,tmp_mu])
    BS_startIndex=BS_startIndex+336
    BS_endIndex=BS_endIndex+336
    iterations=iterations+1
pairBS.reset_index(inplace=True)
pairBS.rename(columns={'train_sub':'sub'},inplace=True)
#pairBS['Analysis']='Between'

In [2]:
pairDS=pd.DataFrame()
DS_startIndex=0
DS_endIndex=168
iterations=0
while DS_endIndex<168001:
    DS_iterat=permDS[DS_startIndex:DS_endIndex]
    for task in taskList:
        tmp=DS_iterat[(DS_iterat['task']==task)]
        tmp_mu=tmp.groupby(['train_sub']).mean()
        tmp_mu['task']=task
        tmp_mu['iteration']=iterations
        pairDS=pd.concat([pairDS,tmp_mu])
    DS_startIndex=DS_startIndex+168
    DS_endIndex=DS_endIndex+168
    iterations=iterations+1
pairDS.reset_index(inplace=True)
pairDS.rename(columns={'train_sub':'sub'},inplace=True)
#pairDS['Analysis']='Between'

In [3]:
pairCV["iteration"] = pairCV.groupby(['sub','task']).cumcount()

In [4]:
iterations=0
startIndex=0
endIndex=48
pairSS=pd.DataFrame()
while endIndex<48001:
    #iterating through each permutationt to recreate the previous plots
    iterat_tmp=permSS[startIndex:endIndex]
    iterat_tmp['iteration']=iterations
    startIndex=startIndex+48
    endIndex=endIndex+48
    iterations=iterations+1
    pairSS=pd.concat([pairSS, iterat_tmp])
pairSS.reset_index(inplace=True)

In [6]:
pairBS.to_csv(thisDir+'output/mL/results/ridge/permutation/BS/reformat_acc.csv',index=False)
pairSS.to_csv(thisDir+'output/mL/results/ridge/permutation/SS/reformat_acc.csv',index=False)
pairDS.to_csv(thisDir+'output/mL/results/ridge/permutation/DS/reformat_acc.csv',index=False)
pairCV.to_csv(thisDir+'output/mL/results/ridge/permutation/CV/reformat_acc.csv',index=False)


In [13]:
#fix CV file names
"""
motor=permCV[['task.1','acc.1','sub.1']]
mem=permCV[['task.2','acc.2','sub.2']]
mix=permCV[['task','acc','sub']]

motor.rename(columns={'task.1':'task', 'acc.1':'acc','sub.1':'sub'},inplace=True)
mem.rename(columns={'task.2':'task', 'acc.2':'acc','sub.2':'sub'},inplace=True)

p_CV=pd.concat([mix,mem,motor])
p_CV.to_csv(thisDir+'output/mL/results/ridge/permutation/CV/acc.csv', index=False)
"""

In [48]:
same_task= pairCV.merge(pairDS,how='left',on=['iteration','sub','task'],suffixes=('','R'))
same_task['acc_diff'] = same_task.acc - same_task.accR.fillna(0)

In [49]:
diff_task= pairSS.merge(pairBS,how='left',on=['iteration', 'train_task','test_task','sub'],suffixes=('','R'))
diff_task['acc_diff'] = diff_task.acc - diff_task.accR.fillna(0)

In [51]:
same_task.to_csv(thisDir+'output/mL/results/ridge/permutation/same_task.csv',index=False)
diff_task.to_csv(thisDir+'output/mL/results/ridge/permutation/diff_task.csv',index=False)

In [7]:
#no generate difference from your results

#all packages and dataframes
import numpy as np
import classification
import os
import sys
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import ptitprince as pt
from matplotlib import colors as mcolors
%matplotlib inline
# Initialization of directory information:
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
restDir=thisDir+'data/mvpa_data/rest/'
outDir = thisDir + 'output/mL/results/ridge/acc/'
subDir=thisDir+'output/subNetwork/'
randDir=thisDir+'output/randomIdx/'
Parcel_params = classification.loadParcelParams('Gordon333',thisDir+'data/Parcel_info/')
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
taskList=['mem','mixed','motor']
netList=['unassign','default', 'visual','fp','dan','van','salience','co','sm','sm-lat', 'auditory','pmn','pon']
feature=[14808,10824,8736, 4620,5264,3151, 484, 4060, 2375,316, 564,45,21]
netRoi=dict([('unassign',14808),('default', 10824),('visual',8736),('fp', 4620),('dan',5264),('van',3151),('salience', 484),('co', 4060),('sm', 2375),('sm-lat', 316),('auditory', 564),('pmn',45),('pon',21)])
sns.set_style("white")
sns.set_context("talk")
# Gordan Palette
enmax_palette = Parcel_params['net_colors']
color_codes_wanted = Parcel_params['networks']
cdict = dict(zip(color_codes_wanted, [mcolors.to_rgba(c) for c in enmax_palette]))
mcolors.get_named_colors_mapping().update(cdict)



#Visualizing change scores from within to between
t_CV=pd.read_csv(outDir+'CV/acc.csv')
t_DS=pd.read_csv(outDir+'DS/acc.csv', usecols=[1,2,3,4])
pairCV=pd.melt(t_CV,id_vars=['sub'],value_vars=['mixed','motor','mem'], var_name='task', value_name='acc')
pairDS=pd.DataFrame()
for task in taskList:
    tmp=t_DS[(t_DS['task']==task)]
    tmp_mu=tmp.groupby(['train_sub']).mean()
    tmp_mu['task']=task
    pairDS=pd.concat([pairDS,tmp_mu])
pairDS.reset_index(inplace=True)
pairDS.rename(columns={'train_sub':'sub'},inplace=True)




#adding thinner lines to different task analysis figure
pairSS=pd.read_csv(outDir+'SS/acc.csv',usecols=[1,2,3,4])
t_BS=pd.read_csv(outDir+'BS/acc.csv',usecols=[1,2,3,4,5])

pairBS=pd.DataFrame()
for task in taskList:
    for test_task in taskList:
        if task==test_task:
            continue
        else:
            tmp=t_BS[(t_BS.train_task==task)&(t_BS.test_task==test_task)]
            tmp_mu=tmp.groupby(['train_sub']).mean()
            tmp_mu['train_task']=task
            tmp_mu['test_task']=test_task
            pairBS=pd.concat([pairBS,tmp_mu])
pairBS.reset_index(inplace=True)
pairBS.rename(columns={'train_sub':'sub'},inplace=True)

pairCV.to_csv(thisDir+'output/mL/results/ridge/permutation/CV/real_reformat_acc.csv',index=False)
pairSS.to_csv(thisDir+'output/mL/results/ridge/permutation/SS/real_reformat_acc.csv',index=False)
pairDS.to_csv(thisDir+'output/mL/results/ridge/permutation/DS/real_reformat_acc.csv',index=False)
pairBS.to_csv(thisDir+'output/mL/results/ridge/permutation/BS/real_reformat_acc.csv',index=False)

In [2]:
same_task= pairCV.merge(pairDS,how='left',on=['sub','task'],suffixes=('','R'))
same_task['acc_diff'] = same_task.acc - same_task.accR.fillna(0)

diff_task= pairSS.merge(pairBS,how='left',on=['train_task','test_task','sub'],suffixes=('','R'))
diff_task['acc_diff'] = diff_task.acc - diff_task.accR.fillna(0)

same_task.to_csv(thisDir+'output/mL/results/ridge/permutation/real_same_task.csv',index=False)
diff_task.to_csv(thisDir+'output/mL/results/ridge/permutation/real_diff_task.csv',index=False)